# LAB 01: MỐI QUAN HỆ CỦA DỮ LIỆU

## 1. Thông tin nhóm

| MSSV     | HỌ VÀ TÊN          |
|:--------:|:-------------------|
| 20127323 | Võ Nhật Tân        |
| 20127447 | Ngô Đức Bảo        |
| 20127275 | Lê Nguyễn Nhật Phú |
| 20127681 | Nguyễn Thiên Phúc  |


## 2. Lấy thông tin từ website

In [34]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta
from bs4 import BeautifulSoup

np.set_printoptions(formatter=dict(float='{:0.2f}'.format))

In [2]:
def pre_process_html(table_html):
    table_html = table_html.replace('>N/A<', '>-<')
    
    table = BeautifulSoup(table_html)

    def show_data_continent(tag):
        if tag.name == 'th':
            return True
        
        return 'data-continent' in tag.attrs and tag.name in ['td', 'th'] and tag['data-continent'] != ""

    for k, th in enumerate(table.find_all(show_data_continent)):
        raw_s = th.text
        th['style'] = "color: yellow"


    for k, th in enumerate(table.find_all('th')):
        th.string = th.text

    return table.prettify()

In [3]:
driver = webdriver.Chrome()
driver.maximize_window()

driver.get("https://www.worldometers.info/coronavirus/#main_table")
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "main_table_countries_today")))

#Get updated date
updated_date = driver.find_element(By.CSS_SELECTOR, 'div#page-top+div')
today = datetime.strptime(updated_date.text, "Last updated: %B %d, %Y, %H:%M %Z")

#Get today data
table_element_today = driver.find_element(By.ID, 'main_table_countries_today')
table_html_today = table_element_today.get_attribute('outerHTML')

#Get yesterday data
driver.find_element(By.ID, 'nav-yesterday-tab').click()
table_element_yesterday = driver.find_element(By.ID, 'main_table_countries_yesterday')
table_html_yesterday = table_element_yesterday.get_attribute('outerHTML')

#Get 2-day ago data
driver.find_element(By.ID, 'nav-yesterday2-tab').click()
table_element_yesterday = driver.find_element(By.ID, 'main_table_countries_yesterday2')
table_html_yesterday2 = table_element_yesterday.get_attribute('outerHTML')

driver.quit()

In [4]:
table_html = pre_process_html('\n'.join([table_html_today, table_html_yesterday, table_html_yesterday2]))
df_list = pd.read_html(table_html, keep_default_na=False)

for day in range(3):
    exp_day = today - timedelta(days=day)
    df_list[day].to_csv(f'./raw_data/{exp_day.strftime("%d-%m-%Y")+"_raw"}.csv', index=False, encoding='UTF-8')

## Tiền xử lý dữ liệu

In [5]:
from os import listdir

In [6]:
def pre_process_file(file_path):
    df = pd.read_csv(file_path)

    #Rename the abnormal column's name
    df.rename(columns={'Tot\xa0Cases/1M pop': 'Tot Cases/1M pop'}, inplace=True)

    #Drop the unnecessary columns
    df.drop(columns=['#', 'Serious,Critical'], inplace=True)

    #Drop the total row
    df = df.drop(labels=[len(df) - 1]).reset_index(drop=True)
    
    #Fill the null value with 0 and replace the NA (-) values by np.nan
    df.fillna(0, inplace=True)
    df.replace(to_replace='-', value=np.nan, inplace=True)

    #Change the type of columns
    columns_name = df.select_dtypes(include='object').columns
    for column_name in columns_name:
        if column_name in ['Country,Other', 'Continent'] : continue
        df[column_name] = df[column_name].astype('float')

    #Check if the data has inconsistent values
    sub_df = df.iloc[1:-1, :]

    cal_tot = round(sub_df['TotalCases'] / sub_df['Population'] * 10**6).replace([np.inf, -np.inf], 0)
    cal_deaths = round(sub_df['TotalDeaths'] / sub_df['Population'] * 10**6).replace([np.inf, -np.inf], 0)

    if not np.all(sub_df['Tot Cases/1M pop'] == cal_tot):
        raise Exception(f"Data in the Tot Cases/ 1M pop column is not consistent")
    
    if not np.all(sub_df['Deaths/1M pop'] == cal_deaths):
        raise Exception(f"Data in the Total Deaths column is not consistent")

    return df

In [7]:
#Apply the preprocess to all the file in raw_data and save them to data folder
files = [f for f in listdir('./raw_data/')]

for file in files:
    url = './raw_data/' + file
    print(url)
    df = pre_process_file(url)
    df.to_csv('./data/' + file.replace('_raw', ''), index=False, encoding='UTF-8')

./raw_data/01-03-2023_raw.csv
./raw_data/02-03-2023_raw.csv
./raw_data/03-03-2023_raw.csv
./raw_data/04-03-2023_raw.csv
./raw_data/26-02-2023_raw.csv
./raw_data/27-02-2023_raw.csv
./raw_data/28-02-2023_raw.csv


## Trực quan hóa dữ liệu

## Sử dụng dữ liệu của khu vực Châu Á trong vòng 7 ngày (từ 26/2/2023 đến 4/3/2023)

In [2]:
df1 = pd.read_csv('./data/26-02-2023.csv')
df2 = pd.read_csv('./data/27-02-2023.csv')
df3 = pd.read_csv('./data/28-02-2023.csv')
df4 = pd.read_csv('./data/01-03-2023.csv')
df5 = pd.read_csv('./data/02-03-2023.csv')
df6 = pd.read_csv('./data/03-03-2023.csv')
df7 = pd.read_csv('./data/04-03-2023.csv')

## Tiền xử lý

In [3]:
df = df1[df1['Country,Other'] == 'Vietnam'].copy()
df['Date'] = '26-02-2023'
df = df.append(df2[df2['Continent'] == 'Asia'])
df.loc[df['Date'].isna(), 'Date'] = '27-02-2023'
df = df.append(df3[df3['Continent'] == 'Asia'])
df.loc[df['Date'].isna(), 'Date'] = '28-02-2023'
df = df.append(df4[df4['Continent'] == 'Asia'])
df.loc[df['Date'].isna(), 'Date'] = '01-03-2023'
df = df.append(df5[df5['Continent'] == 'Asia'])
df.loc[df['Date'].isna(), 'Date'] = '02-03-2023'
df = df.append(df6[df6['Continent'] == 'Asia'])
df.loc[df['Date'].isna(), 'Date'] = '03-03-2023'
df = df.append(df7[df7['Continent'] == 'Asia'])
df.loc[df['Date'].isna(), 'Date'] = '04-03-2023'

df.reset_index(drop=True, inplace=True)

df.head(10)

C:\Users\Nguyen Thien Phuc\AppData\Local\Temp\ipykernel_8364\2925247170.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2[df2['Continent'] == 'Asia'])
C:\Users\Nguyen Thien Phuc\AppData\Local\Temp\ipykernel_8364\2925247170.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df3[df3['Continent'] == 'Asia'])
C:\Users\Nguyen Thien Phuc\AppData\Local\Temp\ipykernel_8364\2925247170.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df4[df4['Continent'] == 'Asia'])
C:\Users\Nguyen Thien Phuc\AppData\Local\Temp\ipykernel_8364\2925247170.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df =

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/ 1M pop,Population,Continent,Date
0,Vietnam,11526901,10.0,43186.0,0.0,10614760.0,1.0,868955.0,116488.0,436.0,85826548.0,867342.0,9.895354e+07,Asia,26-02-2023
1,India,44686371,169.0,530771.0,1.0,44153343.0,140.0,2257.0,31768.0,377.0,918535118.0,653003.0,1.406632e+09,Asia,27-02-2023
2,Japan,33190564,5598.0,72320.0,52.0,21683904.0,1535.0,11434340.0,264288.0,576.0,95014492.0,756576.0,1.255848e+08,Asia,27-02-2023
3,S. Korea,30502904,4026.0,33961.0,15.0,30278033.0,0.0,190910.0,594252.0,662.0,15804065.0,307892.0,5.132990e+07,Asia,27-02-2023
4,Turkey,17042722,0.0,101492.0,0.0,NaN,NaN,NaN,199186.0,1186.0,162743369.0,1902052.0,8.556198e+07,Asia,27-02-2023
5,Vietnam,11526905,4.0,43186.0,0.0,10614763.0,3.0,868956.0,116488.0,436.0,85826548.0,867342.0,9.895354e+07,Asia,27-02-2023
6,Taiwan,10033107,7744.0,17908.0,44.0,9690990.0,23339.0,324209.0,419996.0,750.0,30742304.0,1286903.0,2.388860e+07,Asia,27-02-2023
7,Iran,7567824,122.0,144845.0,3.0,7339283.0,78.0,83696.0,87975.0,1684.0,55034937.0,639771.0,8.602284e+07,Asia,27-02-2023
8,Indonesia,6735780,152.0,160911.0,3.0,6571457.0,270.0,3412.0,24131.0,576.0,114158919.0,408975.0,2.791345e+08,Asia,27-02-2023
9,Malaysia,5042585,190.0,36957.0,0.0,4996409.0,177.0,9219.0,151972.0,1114.0,68079467.0,2051756.0,3.318107e+07,Asia,27-02-2023


In [145]:
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')
df.dtypes

Country,Other               object
TotalCases                   int64
NewCases                   float64
TotalDeaths                float64
NewDeaths                  float64
TotalRecovered             float64
NewRecovered               float64
ActiveCases                float64
Tot Cases/1M pop           float64
Deaths/1M pop              float64
TotalTests                 float64
Tests/ 1M pop              float64
Population                 float64
Continent                   object
Date                datetime64[ns]
dtype: object

In [140]:
df.isna().sum()
df.dropna(inplace=True)

In [132]:
df = df.groupby('Country,Other').apply(lambda x: x.sort_values('Date', ascending=True)).reset_index(drop=True)
df

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/ 1M pop,Population,Continent,Date
0,Vietnam,11526901,10.0,43186.0,0.0,10614760.0,1.0,868955.0,116488.0,436.0,85826548.0,867342.0,9.895354e+07,Asia,2023-02-26
1,India,44686371,169.0,530771.0,1.0,44153343.0,140.0,2257.0,31768.0,377.0,918535118.0,653003.0,1.406632e+09,Asia,2023-02-27
2,Japan,33190564,5598.0,72320.0,52.0,21683904.0,1535.0,11434340.0,264288.0,576.0,95014492.0,756576.0,1.255848e+08,Asia,2023-02-27
3,S. Korea,30502904,4026.0,33961.0,15.0,30278033.0,0.0,190910.0,594252.0,662.0,15804065.0,307892.0,5.132990e+07,Asia,2023-02-27
4,Turkey,17042722,0.0,101492.0,0.0,NaN,NaN,NaN,199186.0,1186.0,162743369.0,1902052.0,8.556198e+07,Asia,2023-02-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,Syria,57467,0.0,3164.0,0.0,54303.0,0.0,0.0,2968.0,163.0,146269.0,7553.0,1.936481e+07,Asia,2023-03-04
297,Timor-Leste,23418,0.0,138.0,0.0,23102.0,0.0,178.0,17101.0,101.0,278529.0,203391.0,1.369429e+06,Asia,2023-03-04
298,Tajikistan,17786,0.0,125.0,0.0,17264.0,0.0,397.0,1786.0,13.0,0.0,0.0,9.957464e+06,Asia,2023-03-04
299,Yemen,11945,0.0,2159.0,0.0,9124.0,0.0,662.0,383.0,69.0,329592.0,10579.0,3.115487e+07,Asia,2023-03-04


In [4]:
numeric_cols = df.select_dtypes(exclude=[object, datetime]).columns
numeric_cols
for col in numeric_cols:
    print(f'Column {col} - Value counts in each bins: ')
    print(df[col].value_counts(bins=3, sort=False))
    print('-'*30)

Column TotalCases - Value counts in each bins: 
(-41169.983, 14898174.667]      277
(14898174.667, 29792835.333]      6
(29792835.333, 44687496.0]       18
Name: TotalCases, dtype: int64
------------------------------
Column NewCases - Value counts in each bins: 
(-14.524999999999999, 4841.333]    284
(4841.333, 9682.667]                 3
(9682.667, 14524.0]                 14
Name: NewCases, dtype: int64
------------------------------
Column TotalDeaths - Value counts in each bins: 
(-509.755, 176939.0]    295
(176939.0, 353857.0]      0
(353857.0, 530775.0]      6
Name: TotalDeaths, dtype: int64
------------------------------
Column NewDeaths - Value counts in each bins: 
(-0.1, 33.0]    290
(33.0, 66.0]      5
(66.0, 99.0]      6
Name: NewDeaths, dtype: int64
------------------------------
Column TotalRecovered - Value counts in each bins: 
(-40763.649, 14720269.667]      271
(14720269.667, 29437152.333]      6
(29437152.333, 44154035.0]       12
Name: TotalRecovered, dtype: int64


- Ta trực quan mối tương quan giữa các biến bằng ma trận correlation

In [144]:
corr = df[numeric_cols].corr()
corr.style.background_gradient(cmap='coolwarm')

,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/ 1M pop,Population
TotalCases,1.000000,0.572155,0.746545,0.511225,0.984965,0.320625,0.500640,0.184955,-0.019027,0.702822,-0.107799,0.471080
NewCases,0.572155,1.000000,0.049323,0.842142,0.506707,0.759375,0.584423,0.381896,-0.000312,0.015273,-0.067617,-0.021436
TotalDeaths,0.746545,0.049323,1.000000,0.077244,0.790775,-0.004487,0.079452,-0.138271,0.000984,0.918752,-0.116698,0.659617
NewDeaths,0.511225,0.842142,0.077244,1.000000,0.393408,0.467936,0.812376,0.254458,0.012257,0.038139,-0.048629,-0.008788
TotalRecovered,0.984965,0.506707,0.790775,0.393408,1.000000,0.338358,0.343650,0.183487,-0.016264,0.748321,-0.105858,0.504899
NewRecovered,0.320625,0.759375,-0.004487,0.467936,0.338358,1.000000,0.061503,0.367792,0.021117,-0.017432,-0.026720,-0.040067
ActiveCases,0.500640,0.584423,0.079452,0.812376,0.343650,0.061503,1.000000,0.099620,-0.022448,0.056927,-0.053776,0.019685
Tot Cases/1M pop,0.184955,0.381896,-0.138271,0.254458,0.183487,0.367792,0.099620,1.000000,0.414272,-0.110529,0.356400,-0.218222
Deaths/1M pop,-0.019027,-0.000312,0.000984,0.012257,-0.016264,0.021117,-0.022448,0.414272,1.000000,-0.066295,0.190732,-0.169002
TotalTests,0.702822,0.015273,0.918752,0.038139,0.748321,-0.017432,0.056927,-0.110529,-0.066295,1.000000,0.108218,0.764748


Từ ma trận correlation trên, ta thấy rằng các cột TotalCases, TotalDeaths, TotalRecovered, ActiveCases, Serious,Critical, Tot Cases/1M pop, Deaths/1M pop, TotalTests, Tests/1M pop có mối tương quan cao với nhau


In [30]:
top_5_Tot_Cases_1M_pop_df = df.sort_values(by=['TotalCases'], ascending=False).iloc[1:26]
top_5_Tot_Cases_1M_pop = top_5_Tot_Cases_1M_pop_df['Country,Other'].unique()
top_5_Tot_Cases_1M_pop

array(['India', 'Japan', 'S. Korea', 'Turkey', 'Vietnam'], dtype=object)

Ta thấy, 5 quốc gia có độ lây lan cao nhất khu vực châu Á là India, Japan, S.Korea, Turkey và Việt Nam
Tốc độ lây lan ở India là cao nhất do dân số đông nhất khu vực Châu Á

In [37]:
#compare the total deaths in the top 5 countries with line chart
fig = px.line(df[df['Country,Other'].isin(top_5_Tot_Cases_1M_pop)], x='Date', y='NewRecovered', color='Country,Other')
fig.show()

In [39]:
fig = px.line(df[df['Country,Other'].isin(top_5_Tot_Cases_1M_pop)], x='Date', y='NewDeaths', color='Country,Other')
fig.show()

Trong vòng 7 ngày, S.Korea là quốc gia có số ca hồi phục nhiều nhất và số ca tử vong giảm dần, có thể nói rằng S.Korea đang đứng đầu trong việc kiểm soát dịch bệnh

In [40]:
asian_countries = ['China', 'India', 'Indonesia', 'Pakistan', 'Bangladesh', 'Japan', 'Brunei',
                   'Philippines', 'Vietnam', 'Iran', 'Turkey', 'Thailand', 'Myanmar', 'S. Korea', 'DPRK',
                   'Iraq', 'Afghanistan', 'Saudi Arabia', 'Uzbekistan', 'Malaysia', 'Nepal', 'Yemen', 
                   'Taiwan', 'Sri Lanka', 'Kazakhstan', 'Syria', 'Cambodia', 'Jordan', 
                   'Azerbaijan', 'UAE', 'Tajikistan', 'Israel', 'Hong Kong', 'Laos', 
                   'Lebanon', 'Kyrgyzstan', 'Singapore', 'Palestine', 'Oman', 'Kuwait', 
                   'Georgia', 'Mongolia', 'Armenia', 'Qatar', 'Bahrain', 'Timor-Leste', 'Cyprus', 'Bhutan', 'Macao', 'Maldives']

df_asia = df[df['Country,Other'].isin(asian_countries)]
# Tạo biểu đồ scatter plot
fig = px.scatter(df_asia, x='TotalCases', y='NewCases', color='Country,Other', log_x=True, log_y=True, size_max=50, title='Total Cases vs. New Cases in Asia countries')
fig.show()

Từ biểu đồ trên, mối tương quan giữa biến NewCases với biến TotalCases là tuyến tính, với số ca nhiễm mới cao nhất là ở các nước: Taiwan, S.Korea và Japan


In [45]:
#total test in asia countries with bar chart
fig = px.bar(df_asia, x='Country,Other', y='TotalTests', color='Country,Other', title='TotalTests')
fig.show()

Về việc thực hiện test covid-19, ta thấy rằng các quốc gia có số ca nhiễm cao nhất thì số ca test cũng cao nhất, đồng thời số ca test cũng cao nhất ở các quốc gia có dân số đông nhất khu vực Châu Á